## Scraping State Crime Rates

In [1]:
# Import libraries

import os
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
import datetime

#### Scrape the data
---

In [7]:
# launch chrome driver
executable_path = {'executable_path': '/Users/prettyvo/Desktop/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [36]:
# creating all links for each state
states = ['Alaska', 'Alabama', 'Arkansas', 'Arizona', 'California', 'Colorado', 'Connecticut', 'District of Columbia', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Iowa', 'Idaho', 'Illinois', 'Indiana', 'Kansas', 'Kentucky', 'Louisiana', 'Massachusetts', 'Maryland', 'Maine', 'Michigan', 'Minnesota', 'Missouri', 'Mississippi', 'Montana', 'Nebraska', 'North Carolina', 'North Dakota', 'New Hampshire', 'New Jersey', 'New Mexico', 'Nevada', 'New York', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhodes Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Virginia', 'Vermont', 'Washington', 'Wisconsin', 'West Virginia', 'Wyoming']
scrape_links = []
for state in states:
    try:
        state = state.replace(' ', '-')
        link = f'https://www.macrotrends.net/states/{state}/murder-homicide-rate-statistics'
    except:
        link = f'https://www.macrotrends.net/states/{state}/murder-homicide-rate-statistics'
    scrape_links.append(link)

In [39]:
# function to scrape the data
def scrape(link):
    browser.visit(link)
    html = browser.html
    state_html = BeautifulSoup(html, 'html.parser')
    browser.click_link_by_id('dataDownload')

In [40]:
run = [scrape(link) for link in scrape_links] # scrape for all csv

In [42]:
browser.quit() # close chrome driver

#### Merge all the Data
---

In [2]:
# get all the files
state_csv = []
for file in os.listdir("../Data/state_crime_rate"):
    if file.endswith(".csv"):
        state_csv.append(file)

In [30]:

def clean_data(path):
    df = pd.read_csv('../Data/state_crime_rate/' + path, skiprows=15)
    col = df.columns.values
    df = df.drop(' US', axis=1)
    try:
        df = df.rename(columns={col[2]:col[3], col[1]:col[2]}).head()
        try:
            df['year'] = [datetime.datetime.strptime(x, '%m/%d/%y').year for x in df['date']]
        except:
            df['year'] = [datetime.datetime.strptime(x, '%Y-%m-%d').year for x in df['date']]
    except: 
        pass
    return df

def merge_df(df1, df2):
    try:
        df2 = df2.drop([' US', 'date'], axis=1)
        df = df1.merge(df2, on ='year', how = 'outer')
    except:
        df = df2
    return df

In [31]:
df_main = pd.DataFrame()
for csv in state_csv:
    df_single = clean_data(csv)
    df_main = merge_df(df_main, df_single)

In [33]:
df_main.to_csv('../Data/state_crime_rate.csv')